In [1]:
{-# LANGUAGE TypeOperators, FlexibleContexts, TypeFamilies, UnicodeSyntax, ScopedTypeVariables #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Random
import Data.Random.Manifold
import Data.Manifold
import Data.Manifold.Web
import Data.Manifold.DifferentialEquation
import Math.LinearMap.Category
import Data.VectorSpace
import Data.Basis (Basis)
import Linear(V2(..), ex, ey, _x, _y)
import Data.Semigroup
import qualified Data.Foldable as Hask
import qualified Control.Monad as Hask
import qualified Control.Comonad.Cofree as Hask
import Control.Lens
:opt no-lint
import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained

From [diagrams](http://projects.haskell.org/diagrams):

In [2]:
import Diagrams.Prelude (p2, circle, (&), (^&), moveTo, opacity, fromVertices, Point(P))

In [3]:
type X = ℝ
type T = ℝ
type U = ℝ
type Ðx'U = ℝ
type Ðt'U = ℝ
type Ðx'Ðt'U = ℝ
type Ðx²'U = ℝ
type x × y = ℝ²
et = ey :: Basis ℝ²

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [4]:
import Graphics.Dynamic.Plot.R2
import Data.Colour.Names
import Data.Colour.Manifold

colourscheme :: Shade' ℝ -> Shade (Colour ℝ)
colourscheme (Shade' u du) = interp (Shade u $ dualNorm du :: Shade ℝ)
 where Just interp = rangeOnGeodesic darkblue orange
 
prettyWebPlot :: PointsWeb ℝ² y -> DynamicPlottable
prettyWebPlot w = plot [ diagramPlot . opacity 0.5 $ fromVertices [P r₁, P r₂]
                       | ((r₁@(V2 x₁ y₁),_),(r₂@(V2 x₂ y₂),_)) <- edg ]
 where edg = webEdges w

In [5]:
colourscheme_heat :: Shade' (U, Ðx'U) -> Shade (Colour ℝ)
colourscheme_heat = cm . factoriseShade
 where cm :: (Shade' U, Shade' Ðx'U) -> Shade (Colour ℝ)
       cm (Shade' u du, _) = interp (Shade u $ dualNorm du :: Shade ℝ)
       Just interp = rangeOnGeodesic darkblue orange

In [6]:
heatEqn :: DifferentialEqn (X × T) Ðt'U (U, Ðx'U)
      -- Shade (X×T, U,Ðx'U) -> Shade' (X×T +> (U,Ðx'U)) -> ?(Shade' (X×T +> (U,Ðx'U)))
heatEqn = constLinearDEqn
    (arr.LinearFunction $
        \((_u, ðx'u), ðt'u)
       -> let ðx'ðx'u = ðt'u
          in LinearMap (V2 (ðx'u, ðx'ðx'u)
                           (ðt'u, 0)) )  -- ðt'ðx'u is unknown
    (arr.LinearFunction $
        \(LinearMap (V2 (ðx'u, ðx'ðx'u)
                        (ðt'u, ðt'ðx'u)))
                  -> ((0, ðx'u), ðt'u) )

In [7]:
type HeatFlow = (U, Ðx'U)

initState_heat :: X -> HeatFlow
initState_heat x = ( u
                   , (- 2 * s*x / cosh (s * (1 - x^2))^2 )
                   )
 where u = tanh (s * (1 - x^2))
       s = 1

plotWindow $ continFnPlot <$> [ fst . initState_heat
                              , snd . initState_heat ]

tf_heat :: Needle X -> Needle T -> PointsWeb (X × T) (Shade' HeatFlow)
tf_heat δx₀ δt₀ = fromWebNodes euclideanMetric
      $  [ (V2 x t, initState_heat x|±|[(0.01,0), (0,0.01)])
         | x<-[xl, xl+δx₀ .. xr], t<-[0] ]
      ++ [ (V2 x t, zeroV|±|[(2, 0), (0, 2)])
         | x<-[xl, xl+δx₀ .. xr], t<-tail[0, δt₀ .. 1] ]
 where (xl,xr) = (0,2)

GraphWindowSpecR2{lBound=-1.333333333333333, rBound=1.333333333333333, bBound=-1.8790981773565483, tBound=1.8790981773565434, xResolution=640, yResolution=480}

In [8]:
startSt_heat = tf_heat 0.125 0.125

-- Hask.toList startSt_heat
-- Hask.toList <$> rescanPDEOnWeb AbortOnInconsistency heatEqn startSt_heat

scrutiniseShade' :: ∀ x . x ~ HeatFlow => Shade' x -> String
scrutiniseShade' (Shade' c (Norm e)) = show c ++ "±" ++ show (arr e :: Needle x +> Needle' x)

tfs_heat = iterateFilterDEqn_static (indicateInconsistencies intersectShade's) id heatEqn startSt_heat
forM_ [Hask.toList tfs_heat ]
  $ \tfs ->
    plotWindow
       [ prettyWebPlot $ head tfs
       , plotLatest [ plot (fmap colourscheme_heat tfi)
                       & legendName ("i = "++show i)
                    | (i,tfi) <- zip [0..] tfs ] ]
Hask.mapM_ (\st -> mapM_ (\((p,y),(q,z)) -> putStrLn $ show p ++ ":\t" ++ prettyShowShade' y ++ "\n" ++ show q ++ ":\t" ++ prettyShowShade' z) (webEdges st) >> print()) tfs_heat

V2 2.0 0.625:	(0,0)|±|[(2,0),(0,2)]
V2 2.0 0.75:	(0,0)|±|[(2,0),(0,2)]
V2 2.0 0.625:	(0,0)|±|[(2,0),(0,2)]
...
...
...
V2 0.75 0.0:	(0.41,-1.246)|±|[(0.04,1e-3),(0,0.010)]
V2 0.75 0.0:	(0.41,-1.246)|±|[(0.04,1e-3),(0,0.010)]
V2 0.875 0.0:	(0.23,-1.657)|±|[(0.05,1e-3),(-0.00,11e-3)]
()

In [15]:
import Control.Monad.Trans.Except
findErr :: (Hask.Monad m, Hask.Foldable m) => Hask.Cofree m a -> m ()
findErr (a Hask.:< q) = case Hask.toList q of
        [] -> const () <$> q
        l  -> foldr1 (>>) $ findErr<$>l

case runExcept $ findErr tfs_heat of Left e -> print e

PropagationInconsistency {_inconsistentPropagatedData = [(V2 1.625 0.125,(-0.93,0e9)|±|[(0.07,0)]),(V2 1.75 0.0,(-0.9,-1.17)|±|[(0.4,-0.02),(0,0.18)]),(V2 1.5 0.0,(-1,-0)|±|[(3,8),(0,7)])], _inconsistentAPrioriData = (-0.93,-0.454)|±|[(0.04,-7e-3),(0.00,0.011)]}

![Initial state from which to start refining solution of the heat PDE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/PDE-solution-filter/HeatEqn-InitialState.png)